In [1]:
import numpy as np
import math
import os

def pro2label(label,pro):
    cnt=0
    for i in range(pro.shape[0]):
        if np.argmax(pro[i,:])==label[i]:
            cnt+=1
    return cnt/pro.shape[0]

In [2]:
root="VAL/"
files=os.listdir(root)
csv=[]
npy=[]
for file in files:
    if file.count(".csv")>0:
        csv.append(root+file)
    elif file=="RULE" or file=="val_files.txt" or file=="model123_index":
        continue
    else:
        subfiles=os.listdir(root+file+"/")
        for subfile in subfiles:
            if subfile.count("acc")>0:
                npy.append(root+file+"/"+subfile)
npy.sort()
csv.sort()        

In [3]:
true_labels=[]
ids=[]
f=open(root+"val_files.txt")
lines=f.readlines()
for line in lines:
    s=line.split("/")[-1].split(".txt")[0].split("_")
    true_labels.append(int(s[1]))
    ids.append(int(s[0]))
f.close()

In [4]:
def single(data):
    assert data.shape==(50000,9)
    cnt=0
    for i in range(50000):
        temp=data[i,:]
        if np.argmax(temp)+1==true_labels[i]:
            cnt+=1
    return cnt/50000
def csv2npy(file,num=50000):
    f=open(file)
    return_npy=np.zeros([num,9])
    lines=f.readlines()
    cnt=0
    for line in lines:
        s=line.split(",")
        for k in range(9):
            return_npy[cnt,k]=float(s[k])
        cnt+=1
    f.close()
    return return_npy
def npy2npy(file,num=50000):
    data=np.load(file)
    assert data.shape==(num,9)
    return data
def bn(temp):
    shp=temp.shape
    MEAN=np.mean(temp,axis=1)
    STD=np.std(temp,axis=1)
    for i in range(shp[0]):
        temp[i,:]=(temp[i,:]-MEAN[i])/(STD[i]+1e-10)
    temp0=np.zeros([temp.shape[0],9])
    for j in range(temp.shape[0]):
        SUM=0.0
        for i in range(9):
            SUM+=math.exp(float(temp[j,i]))
        for i in range(9):
            temp0[j,i]=math.exp(float(temp[j,i]))/SUM
    return temp0


In [5]:
models=[]
model_names=[]
id2index={}
cnt=0
for i in ids:
    id2index[i]=cnt
    cnt+=1
for file in csv:
    s=file.split("[")[1].split("_")[0]
    temp_id=np.load(root+"model123_index/name_id"+str(s)+".npy")
    temp=np.zeros([50000,9])
    temp_npy=csv2npy(file)
    cnt=0
    for i in temp_id:
        index=id2index[i]
        temp[index,:]=temp_npy[cnt,:]
        cnt+=1
    models.append(temp)
    model_names.append(file)
for file in npy:
    models.append(npy2npy(file))
    model_names.append(file)
cnt=0
acc_model=[]
print("the acc on the val dataset in 28 CNN models:")
for model in models:
    acc=single(model)
    acc_model.append([acc,model_names[cnt],model])
    print("val acc:",acc,model_names[cnt],cnt)
    cnt+=1

the acc on the val dataset in 28 CNN models:
val acc: 0.6838 VAL/csvO[1_18].csv 0
val acc: 0.67364 VAL/csvO[2_18].csv 1
val acc: 0.67008 VAL/csvO[3_31].csv 2
val acc: 0.69878 VAL/101/VAL_csv_0716_0_best_acc.npy 3
val acc: 0.6614 VAL/50_5f/VAL_csv_0716_1_best_acc.npy 4
val acc: 0.71152 VAL/50_5f/VAL_csv_0716_2_best_acc.npy 5
val acc: 0.7133 VAL/50_5f/VAL_csv_0716_3_best_acc.npy 6
val acc: 0.70272 VAL/50_5f/VAL_csv_0716_4_best_acc.npy 7
val acc: 0.72154 VAL/50_5f/VAL_csv_0716_5_best_acc.npy 8
val acc: 0.6733 VAL/71/VAL_csv_0716_0_best_acc.npy 9
val acc: 0.71006 VAL/A/VAL_csv_0716_1_best_acc.npy 10
val acc: 0.6971 VAL/A/VAL_csv_0716_2_best_acc.npy 11
val acc: 0.7095 VAL/A/VAL_csv_0716_3_best_acc.npy 12
val acc: 0.68812 VAL/A/VAL_csv_0716_4_best_acc.npy 13
val acc: 0.67054 VAL/A/VAL_csv_0716_5_best_acc.npy 14
val acc: 0.67014 VAL/A/VAL_csv_0716_6_best_acc.npy 15
val acc: 0.66088 VAL/A/VAL_csv_0716_7_best_acc.npy 16
val acc: 0.66436 VAL/A/VAL_csv_0716_8_best_acc.npy 17
val acc: 0.70258 VAL/

In [6]:
def ruletxt2num(file):
    data=np.zeros([50000,9])
    f=open(file)
    lines=f.readlines()
    cnt=0
    for line in lines:
        s=line.split(",")
        for i in range(9):
            data[cnt][i]=float(s[i+1])
        cnt+=1
    f.close()
    return data

rule_names=[]
temp=os.listdir(root+"RULE/")
temp.sort()
for subpath in temp:
    rule_names.append(root+"RULE/"+subpath+"/")

numvalues=[]
for rule_name in rule_names:
    files=os.listdir(rule_name)
    if "id2num_val.txt" in files:
        temp_num=ruletxt2num(rule_name+"id2num_val.txt")
        numvalues.append(temp_num)
    elif "id2num_val.npy" in files:
        numvalues.append(npy2npy(rule_name+"id2num_val.npy"))
    else:
        print("error!",rule_name,files)

In [7]:
acc_rule=[]
print("the acc on the val dataset in 7 rules and the LGB base model:")
for i,numvalue in enumerate(numvalues):
    acc=single(numvalue)
    acc_rule.append([acc,rule_names[i],numvalue])
    print("val acc:",acc,rule_names[i],i)

the acc on the val dataset in 7 rules and the LGB base model:
val acc: 0.7669 VAL/RULE/cxq/ 0
val acc: 0.77696 VAL/RULE/cxq2/ 1
val acc: 0.6181 VAL/RULE/cxq_lgb/ 2
val acc: 0.71704 VAL/RULE/day/ 3
val acc: 0.7062 VAL/RULE/hour/ 4
val acc: 0.7182 VAL/RULE/lyg_day/ 5
val acc: 0.68716 VAL/RULE/lyg_holiday/ 6
val acc: 0.68572 VAL/RULE/totaltime/ 7


In [8]:
def confusion(data):    #get confusion matrix
    assert data.shape==(50000,9)
    label_right=np.zeros([9])
    label_cnt=np.zeros([9])
    cnt=0
    for i in range(50000):
        temp=data[i,:]
        label_cnt[true_labels[i]-1]+=1
        if np.argmax(temp)+1==true_labels[i]:
            cnt+=1
            label_right[true_labels[i]-1]+=1
    acc_label=[]
    for i in range(9):
        acc_label.append(format(label_right[i]/label_cnt[i],".4f"))
    return acc_label

In [9]:
num=50000
model_num=len(acc_model)+len(acc_rule)
all_data=np.zeros([model_num,num,9])
data=np.zeros([num,model_num*9])
labels=np.zeros([num,9])
labels_1d=np.zeros([num])
for i in range(num):
    labels[i,true_labels[i]-1]=1
    labels_1d[i]=true_labels[i]
cnt=0
for _,__,model in acc_model:
    if _<0.5:
        continue
    all_data[cnt,:,:]=bn(model)
    cnt+=1
for _,__,rule in acc_rule:
    # bn
    all_data[cnt,:,:]=bn(rule)
    cnt+=1
for i in range(num):
    for j in range(model_num):
        data[i,(j*9):((j+1)*9)]=all_data[j,i,:]

#######################################
VAL_X=data
VAL_Y=labels_1d
#######################################

In [11]:
train_names=[]
modelname2index={}
cnt=0
for _,name,_ in acc_model:
    if "/50_5f/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                modelname2index["50_5f"+str(i)]=cnt
                cnt+=1
    elif "squeezenet" in name:
        modelname2index["squeezenet"]=cnt
        cnt+=1
    elif "/101/" in name:
        modelname2index["101"]=cnt
        cnt+=1
    elif "/71/" in name:
        modelname2index["71"]=cnt
        cnt+=1
    elif "/A/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                modelname2index["A"+str(i)]=cnt
                cnt+=1
    elif "/B/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                modelname2index["B"+str(i)]=cnt
                cnt+=1
    elif "/C/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                modelname2index["C"+str(i)]=cnt
                cnt+=1
    else:
        for i in [1,2,3]:
            if "["+str(i) in name:
                modelname2index[str(i)]=cnt
                cnt+=1
    train_names.append(name)
print(train_names,len(train_names),model_num)

['VAL/csvO[1_18].csv', 'VAL/csvO[2_18].csv', 'VAL/csvO[3_31].csv', 'VAL/101/VAL_csv_0716_0_best_acc.npy', 'VAL/50_5f/VAL_csv_0716_1_best_acc.npy', 'VAL/50_5f/VAL_csv_0716_2_best_acc.npy', 'VAL/50_5f/VAL_csv_0716_3_best_acc.npy', 'VAL/50_5f/VAL_csv_0716_4_best_acc.npy', 'VAL/50_5f/VAL_csv_0716_5_best_acc.npy', 'VAL/71/VAL_csv_0716_0_best_acc.npy', 'VAL/A/VAL_csv_0716_1_best_acc.npy', 'VAL/A/VAL_csv_0716_2_best_acc.npy', 'VAL/A/VAL_csv_0716_3_best_acc.npy', 'VAL/A/VAL_csv_0716_4_best_acc.npy', 'VAL/A/VAL_csv_0716_5_best_acc.npy', 'VAL/A/VAL_csv_0716_6_best_acc.npy', 'VAL/A/VAL_csv_0716_7_best_acc.npy', 'VAL/A/VAL_csv_0716_8_best_acc.npy', 'VAL/B/VAL_csv_0716_2_best_acc.npy', 'VAL/C/VAL_csv_0716_10_best_acc.npy', 'VAL/C/VAL_csv_0716_1_best_acc.npy', 'VAL/C/VAL_csv_0716_2_best_acc.npy', 'VAL/C/VAL_csv_0716_5_best_acc.npy', 'VAL/C/VAL_csv_0716_6_best_acc.npy', 'VAL/C/VAL_csv_0716_7_best_acc.npy', 'VAL/C/VAL_csv_0716_8_best_acc.npy', 'VAL/C/VAL_csv_0716_9_best_acc.npy', 'VAL/squeezenet/VAL_c

In [12]:
#######################################测试数据读取
root="TEST/"
files=os.listdir(root)
csv=[]
npy=[]
for file in files:
    if file.count(".csv")>0:
        csv.append(root+file)
    elif "RULE" in file:
        continue
    else:
        subfiles=os.listdir(root+file+"/")
        for subfile in subfiles:
            if "loss" not in subfile:
                npy.append(root+file+"/"+subfile)

temp=[]
temp1=[]
for name in npy:
    if "/50_5f/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                temp1.append(name)
                temp.append("50_5f"+str(i))
    elif "squeezenet" in name:
        temp1.append(name)
        temp.append("squeezenet")
    elif "/101/" in name:
        temp1.append(name)
        temp.append("101")
    elif "/71/" in name:
        temp1.append(name)
        temp.append("71")
    elif "/A/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                temp1.append(name)
                temp.append("A"+str(i))
    elif "/B/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                temp1.append(name)
                temp.append("B"+str(i))
    elif "/C/" in name:
        for i in range(11):
            if "_"+str(i)+"_" in name:
                temp1.append(name)
                temp.append("C"+str(i))
    else:
        for i in [1,2,3]:
            if "["+str(i) in name:
                temp1.append(name)
                temp.append(str(i))
for name in csv:
    for i in [1,2,3]:
        if "["+str(i) in name:
            temp1.append(name)
            temp.append(str(i))
test_files=[0 for i in range(len(temp))]
test_model_name=[]
for i,name in enumerate(temp):
    test_model_name.append(name)
    test_files[modelname2index[name]]=temp1[i]

test_model_num=len(test_files)+len(rule_names)
models=np.zeros([test_model_num,100000,9])
cnt=0
for file in test_files:
    if ".csv" in file:
        npy=csv2npy(file,100000)
    else:
        npy=npy2npy(file,100000)
    models[cnt,:,:]=bn(npy)
    cnt+=1
for rule_name in rule_names:
    s=rule_name.split("/")[-2]
    files=os.listdir(root+"RULE/"+s+"/")
    npy=npy2npy(root+"RULE/"+s+"/"+files[0],100000)
    models[cnt,:,:]=bn(npy)
    cnt+=1

In [13]:
num=100000
model_num=test_model_num
data=np.zeros([num,model_num*9])
for i in range(num):
    for j in range(model_num):
        data[i,(j*9):((j+1)*9)]=models[j,i,:]

if not os.path.exists("SUBMISSION"):
    os.makedirs("SUBMISSION")
###############################
TEST_X=data
###############################

In [14]:
#36个模型的输出已经全部读取并归一化
#VAL_X 的维度为[50000,36*9],二维矩阵，36个模型在验证集的输出拼接而成
#VAL_Y 的维度为[50000],一维向量，验证集样本的类别标注，数值从1到9
#TEST_X 的维度为[100000,36*9],二维矩阵，36个模型在测试集的输出拼接而成
#接下来只需利用VAL_X、VAL_Y、TEST_X 三个变量进行集成学习训练lgb即可
print(VAL_X.shape,VAL_Y.shape,TEST_X.shape)

(50000, 324) (50000,) (100000, 324)


In [15]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
params = {
'boosting_type': 'dart',
'objective': 'multiclassova',  
'num_class': 9,  
'metric': 'multi_error', 
'num_leaves': 63,
'learning_rate': 0.01,
'feature_fraction': 0.9,
'bagging_fraction': 0.9,
'bagging_seed':0,
'bagging_freq': 1,
'verbose': -1,
'reg_alpha':1,
'reg_lambda':2,
'lambda_l1': 0,
'lambda_l2': 1,
'num_threads': 16,
'drop_rate':0.95,
}
############################################################################ 5万验证集拿出4万训练并留1万数据验证（为了找出最佳的训练轮数）
X_train, X_valid, y_train, y_valid = train_test_split(VAL_X, VAL_Y-1, test_size=0.2, random_state=2019)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_evals = lgb.Dataset(X_valid, y_valid , reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3250,    #3250轮是试验得出，val loss不再下降
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                #early_stopping_rounds=300,
                verbose_eval=50,
                )
train_pro = gbm.predict(X_train, num_iteration=gbm.best_iteration)
val_pro = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
print ('train_acc：',pro2label(y_train,train_pro),'val_acc：',pro2label(y_valid,val_pro))

[50]	train's multi_error: 0.138175	valid's multi_error: 0.1719
[100]	train's multi_error: 0.11855	valid's multi_error: 0.1538
[150]	train's multi_error: 0.113725	valid's multi_error: 0.1518
[200]	train's multi_error: 0.110575	valid's multi_error: 0.1515
[250]	train's multi_error: 0.10865	valid's multi_error: 0.1504
[300]	train's multi_error: 0.106125	valid's multi_error: 0.1501
[350]	train's multi_error: 0.103375	valid's multi_error: 0.1502
[400]	train's multi_error: 0.101125	valid's multi_error: 0.1496
[450]	train's multi_error: 0.097925	valid's multi_error: 0.1497
[500]	train's multi_error: 0.095575	valid's multi_error: 0.1494
[550]	train's multi_error: 0.0937	valid's multi_error: 0.1492
[600]	train's multi_error: 0.09095	valid's multi_error: 0.1482
[650]	train's multi_error: 0.087875	valid's multi_error: 0.1476
[700]	train's multi_error: 0.084925	valid's multi_error: 0.1477
[750]	train's multi_error: 0.081425	valid's multi_error: 0.147
[800]	train's multi_error: 0.07905	valid's mult

In [16]:
############################################################################ 5万验证集全部用于训练
lgb_train = lgb.Dataset(VAL_X, VAL_Y-1)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3250+100,
                valid_sets=[lgb_train],
                valid_names=['train'],
                verbose_eval=100,
                )

[100]	train's multi_error: 0.1232
[200]	train's multi_error: 0.11638
[300]	train's multi_error: 0.11238
[400]	train's multi_error: 0.10748
[500]	train's multi_error: 0.10236
[600]	train's multi_error: 0.09754
[700]	train's multi_error: 0.09242
[800]	train's multi_error: 0.08686
[900]	train's multi_error: 0.08208
[1000]	train's multi_error: 0.07728
[1100]	train's multi_error: 0.07302
[1200]	train's multi_error: 0.06902
[1300]	train's multi_error: 0.06558
[1400]	train's multi_error: 0.06152
[1500]	train's multi_error: 0.05774
[1600]	train's multi_error: 0.05372
[1700]	train's multi_error: 0.05054
[1800]	train's multi_error: 0.0472
[1900]	train's multi_error: 0.04344
[2000]	train's multi_error: 0.03976
[2100]	train's multi_error: 0.03642
[2200]	train's multi_error: 0.03328
[2300]	train's multi_error: 0.02986
[2400]	train's multi_error: 0.02708
[2500]	train's multi_error: 0.02392
[2600]	train's multi_error: 0.02128
[2700]	train's multi_error: 0.01942
[2800]	train's multi_error: 0.01742
[29

In [17]:
############################################################################测试并生成SUBMISSION
f=open("SUBMISSION/lgb_dart095_8586_2ruleall_cxq2ruleall.txt","w")
ans=gbm.predict(TEST_X, num_iteration=3350)
for i in range(100000):
    f.write(str(i).zfill(6)+"\t"+str(int(np.argmax(ans[i])+1)).zfill(3)+"\n")
f.close()

In [18]:
gbm.save_model("lgb_dart095_8586.txt")